In [1]:
import nltk
import numpy as np
import pandas as pd
import pickle
import string
from sklearn.preprocessing import OneHotEncoder


In [2]:
%matplotlib inline

In [3]:
pubmed = pickle.load( open( "./Pubmed/bins/pubmed_1998.p", "rb" ), encoding="latin1" )

In [4]:
pubmed.head(10)

,pubmedid,year,title,jounal,lang
600465,11655160,1998,Controversies in transfusion medicine: directe...,Transfusion,eng
600787,11648630,1998,W v. Egdell.,The all England law reports,eng
601826,12293749,1998,AIDS as a political issue: working with the se...,Community development journal,eng
602678,12158206,1998,"[Decree No. 70-89 of 23 November 1989, the For...","Diario de Centro América (Guatemala, Guatemala...",spa
630408,11660372,1998,"Refusal of medical treatment in ""captive"" circ...",The Canadian bar review,eng
630409,11652468,1998,The challenge of biotechnology.,Yale law & policy review,eng
630410,11655150,1998,Regulating human gene therapy.,West Virginia law review,eng
630411,11660404,1998,Informed consent to psychotherapy: current pra...,Law and psychology review,eng
630412,11660412,1998,Treatment dilemmas for imperiled newborns: why...,Southern California law review,eng
630413,11645795,1998,Governmental regulation of the investigation o...,Minerva,eng


In [5]:
values = dict()

In [6]:
for i, row in pubmed.iterrows():
    for c in row['title'].lower():
        if c not in values:
            values[c] =1
        

In [7]:
len(values)

101

In [8]:
values[' ']

1

In [9]:
# word to id dict
chr_to_id = {token: idx for idx, token in enumerate(values)}
id_to_chr = {idx: token for idx, token in enumerate(values)}


# convert token lists to token-id lists, e.g. [[1, 2], [2, 2]] here
#token_ids = [[word_to_id[token] for token in tokens_doc] for tokens_doc in tokens_docs]

In [10]:
len(chr_to_id)

101

In [11]:
# generate a continuous text woth all the titles. 
pubmed_corpus = list()
for i, row in pubmed.iterrows():
    pubmed_corpus.append(row['title'].lower())

pubmed_corpus = ' '.join(pubmed_corpus)

In [12]:
 pubmed_corpus[0:0 + 50]

'controversies in transfusion medicine: directed bl'

In [13]:
n_chars_corpus = len(pubmed_corpus)

In [14]:
n_chars_corpus

39772577

In [15]:
len(range(0, n_chars_corpus - 50, 20))

1988627

In [16]:
# float window 5 to 1
n_chars_corpus = len(pubmed_corpus)
max_len = 50
overlap = 20
dataX = []
dataY = []
for i in range(0, n_chars_corpus - max_len, overlap):
	seq_in = pubmed_corpus[i:i + max_len]
	seq_target = pubmed_corpus[i + max_len]
	dataX.append([chr_to_id[char] for char in seq_in])
	dataY.append(chr_to_id[seq_target])
n_patterns = len(dataX)
print( "Total Patterns: {}".format( n_patterns))


Total Patterns: 1988627


In [17]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, Activation
from keras.layers import LSTM
from keras.datasets import imdb
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [18]:
X = np.zeros((n_patterns, max_len, len(chr_to_id)), dtype=np.bool)
y = np.zeros((n_patterns, len(chr_to_id)), dtype=np.bool)



# vectorize in LSTM format   
for idx,title in enumerate(dataX):
    for jdx, c in enumerate(title):
        X[idx, jdx, c] = 1
    y[idx, dataY[idx]] = 1
        
        
    
    

In [19]:
X.shape

(1988627, 50, 101)

In [20]:
y.shape

(1988627, 101)

In [21]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(chr_to_id))))
model.add(Dropout(0.2))
model.add(Dense(len(chr_to_id)))
model.add(Activation('softmax'))

#optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Build model...


In [22]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [23]:
from keras.callbacks import  TensorBoard

In [24]:
tensorboard = TensorBoard(log_dir="logs/1")

In [25]:
tensorboard.set_model(model)


In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256)               366592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 101)               25957     
_________________________________________________________________
activation_1 (Activation)    (None, 101)               0         
Total params: 392,549
Trainable params: 392,549
Non-trainable params: 0
_________________________________________________________________


In [28]:
print('-' * 50)
fileName= '1998_char'
filepath = fileName+"-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint, tensorboard]
model.fit(X, y, batch_size=256, nb_epoch=20, callbacks=callbacks_list)

--------------------------------------------------


/home/ccorbi/anaconda3/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/20
1988627/1988627 [==============================] - 935s - loss: 2.1028   
Epoch 2/20
1988627/1988627 [==============================] - 933s - loss: 1.6683   
Epoch 3/20
1988627/1988627 [==============================] - 927s - loss: 1.5281   
Epoch 4/20
1988627/1988627 [==============================] - 926s - loss: 1.4587   
Epoch 5/20
1988627/1988627 [==============================] - 925s - loss: 1.4160   
Epoch 6/20
1988627/1988627 [==============================] - 927s - loss: 1.3853   
Epoch 7/20
1988627/1988627 [==============================] - 926s - loss: 1.3634   
Epoch 8/20
1988627/1988627 [==============================] - 928s - loss: 1.3447   
Epoch 9/20
1988627/1988627 [==============================] - 927s - loss: 1.3302   
Epoch 10/20
1988627/1988627 [==============================] - 927s - loss: 1.3177   
Epoch 11/20
1988627/1988627 [==============================] - 928s - loss: 1.3071   
Epoch 12/20
1988627/1988627 [==============================] - 

In [29]:
model.fit(X, y, batch_size=256, nb_epoch=200, callbacks=callbacks_list)

/home/ccorbi/anaconda3/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/200
1988627/1988627 [==============================] - 922s - loss: 1.2486   
Epoch 2/200
1988627/1988627 [==============================] - 922s - loss: 1.2451   
Epoch 3/200
1988627/1988627 [==============================] - 922s - loss: 1.2417   
Epoch 4/200
1988627/1988627 [==============================] - 921s - loss: 1.2386   
Epoch 5/200
1988627/1988627 [==============================] - 923s - loss: 1.2354   
Epoch 6/200
1988627/1988627 [==============================] - 921s - loss: 1.2334   
Epoch 7/200
1988627/1988627 [==============================] - 923s - loss: 1.2303   
Epoch 8/200
1988627/1988627 [==============================] - 922s - loss: 1.2275   
Epoch 9/200
1988627/1988627 [==============================] - 922s - loss: 1.2262   
Epoch 10/200
1988627/1988627 [==============================] - 922s - loss: 1.2234   
Epoch 11/200
1988627/1988627 [==============================] - 922s - loss: 1.2213   
Epoch 12/200
1988627/1988627 [======================

1988627/1988627 [==============================] - 924s - loss: 1.1661   
Epoch 67/200
1988627/1988627 [==============================] - 922s - loss: 1.1654   
Epoch 68/200
1988627/1988627 [==============================] - 923s - loss: 1.1649   
Epoch 69/200
1988627/1988627 [==============================] - 922s - loss: 1.1641   
Epoch 70/200
1988627/1988627 [==============================] - 922s - loss: 1.1638   
Epoch 71/200
1988627/1988627 [==============================] - 923s - loss: 1.1633   
Epoch 72/200
1988627/1988627 [==============================] - 922s - loss: 1.1637   
Epoch 73/200
1988627/1988627 [==============================] - 922s - loss: 1.1630   
Epoch 74/200
1988627/1988627 [==============================] - 923s - loss: 1.1621   
Epoch 75/200
1988627/1988627 [==============================] - 923s - loss: 1.1617   
Epoch 76/200
1988627/1988627 [==============================] - 923s - loss: 1.1607   
Epoch 77/200
1988627/1988627 [==========================

1988627/1988627 [==============================] - 923s - loss: 1.1407   
Epoch 140/200
1988627/1988627 [==============================] - 922s - loss: 1.1404   
Epoch 141/200
1988627/1988627 [==============================] - 922s - loss: 1.1405   
Epoch 142/200
1988627/1988627 [==============================] - 922s - loss: 1.1401   
Epoch 143/200
1988627/1988627 [==============================] - 924s - loss: 1.1397   
Epoch 144/200
1988627/1988627 [==============================] - 921s - loss: 1.1402   
Epoch 145/200
1988627/1988627 [==============================] - 923s - loss: 1.1390   
Epoch 146/200
1988627/1988627 [==============================] - 922s - loss: 1.1385   
Epoch 147/200
1988627/1988627 [==============================] - 923s - loss: 1.1384   
Epoch 148/200
1988627/1988627 [==============================] - 924s - loss: 1.1379   
Epoch 149/200
1988627/1988627 [==============================] - 923s - loss: 1.1370   
Epoch 150/200
1988627/1988627 [===============

In [31]:
!pwd

/home/ccorbi/Work/Beagle/training/LSTM


In [42]:
import random


In [44]:
 random.randint(0, len(word_to_id) )

14987

In [ ]:
for i in  random.randint(0, len(word_to_id) )

In [51]:
word_to_id['genetics']

37609

In [59]:
# my seed
seed = ['a','novel', 'study', 'on', 'human' ]

In [60]:

title_ids = return_ids(seed)



In [61]:
title_ids

[35340, 44721, 42855, 15283, 20841]

In [62]:
x = np.zeros((1, max_len, len(word_to_id)))
for t, word in enumerate(title_ids):
    x[0, t, word] =1 

In [66]:
pred = model.predict(x)[0]

In [68]:
pred.shape

(65659,)

In [69]:
temperature = 1
preds = np.asarray(pred).astype('float64')
preds = np.log(preds) / temperature
exp_preds = np.exp(preds)
preds = exp_preds / np.sum(exp_preds)
probas = np.random.multinomial(1, preds, 1)
np.argmax(probas)

38887

In [70]:
id_to_word[38887]

u'large'

In [71]:
seed2 = ['novel', 'study', 'on', 'human','large' ]
title_ids = return_ids(seed2)

x = np.zeros((1, max_len, len(word_to_id)))
for t, word in enumerate(title_ids):
    x[0, t, word] =1 
pred = model.predict(x)[0]
temperature = 1
preds = np.asarray(pred).astype('float64')
preds = np.log(preds) / temperature
exp_preds = np.exp(preds)
preds = exp_preds / np.sum(exp_preds)
probas = np.random.multinomial(1, preds, 1)
id_to_word[np.argmax(probas)]

u'heart'

In [ ]:
start_index = random.randint(0, len(raw_text) - max_len - 1)
#len(word_to_id)
import gc
for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    sentence = raw_text[start_index: start_index + maxlen]
    generated += ' '.join([value for value in sentence])
    print('----- Generating with seed: "' + ' '.join([value for value in sentence]) + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x = np.zeros((1, max_len, len(word_to_id)))
        for t, word in enumerate(sentence):
            x[0, t, char_indices[word]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char + " "
        sentence = sentence[1:]
        sentence.append(next_char)

        sys.stdout.write(" " + next_char)
        sys.stdout.flush()
    print()
gc.collect()